### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = fernet_cipher('0')
samples_count = 500000
text_length = 10

set_characters(letters)
model_path = f'models/{encrypt.name}_{len(letters)}x{text_length}_best_model.h5'

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, activation='relu', return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Bidirectional(GRU(64, activation='relu')))
model.add(Dense(text_length, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 100, 256)          99840     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               123264    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 224,394
Trainable params: 224,394
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=10, callbacks=callbacks)

Train on 450000 samples, validate on 50000 samples
Epoch 1/10
450000/450000 [==============================] - 6452s 14ms/step - loss: 321.8115 - acc: 0.1000 - val_loss: 320.6026 - val_acc: 0.1021

Epoch 00001: val_acc improved from -inf to 0.10210, saving model to models/fernet_cipher_62x10_best_model.h5
Epoch 2/10
450000/450000 [==============================] - 6450s 14ms/step - loss: 320.9975 - acc: 0.0998 - val_loss: 320.3492 - val_acc: 0.1071

Epoch 00002: val_acc improved from 0.10210 to 0.10706, saving model to models/fernet_cipher_62x10_best_model.h5
Epoch 3/10
450000/450000 [==============================] - 6432s 14ms/step - loss: 320.8053 - acc: 0.1000 - val_loss: 320.4433 - val_acc: 0.1046

Epoch 00003: val_acc did not improve from 0.10706


### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HelloWorld')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print(f'\nTest cipher vs encrypted prediction:\n{test_cyhper}\n{encrypt(prediction)}')

Model predicted that original text is: vvuuuuuuuu

Test cipher vs encrypted prediction:
gAAAAABcB9c2ZYS-t1augSIUb8hO8YG6WYpNbM9xHp-gQ-SeNTX9FEUrE3hVlwK3BzCcbC67Ri49Nl_AmV2BJMcGkhOG9AgStQ==
gAAAAABcB9c2imYAwmS9sEDQ5d7bQPAKEE-_OJjMskis5_UvUdQazYx0Sn76pcFRV43X6OllANO3BCE_IUIn53hNBKIuWQjINw==
